In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.9 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [2]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('/kaggle/input/vseros-georate/reviews_sentiment.csv')

In [3]:
reviews

id                                               text  \
0       43591  Мармелад в целом неплохой, но цены завышены, м...   
1       43591  Не нравится, что товар выложен открыто, слишко...   
2       43591  Часто попадается сухой мармелад, дубовый впере...   
3       43591  Персонал был одет в костюмы пиратов, а ассорти...   
4       43591  Вкусный мармелад с широким ассортиментом форм,...   
...       ...                                                ...   
440077   2669  Врачи проявили очень высокий уровень внимания ...   
440078   2669  Никогда и нигде не было проблем с забором кров...   
440079   2669  Советский Союз расположен почти в центре Москв...   
440080   2669  Отличный медицинский персонал! Доброжелательны...   
440081   2669  Пришла в эту женскую консультацию и попала на ...   

        Very Negative  Negative   Neutral  Positive  Very Positive  
0            0.020024  0.505437  0.024622  0.435573       0.014343  
1            0.049843  0.900585  0.020751  0.017807       0.011014  
2            0.049585  0.826169  0.076607  0.027675       0.019963  
3            0.018870  0.018118  0.014912  0.657993       0.290108  
4            0.021226  0.014071  0.904885  0.035124       0.024694  
...               ...       ...       ...       ...            ...  
440077       0.018171  0.017703  0.029441  0.689333       0.245353  
440078       0.486953  0.431961  0.037729  0.024860       0.018498  
440079       0.099327  0.620608  0.172153  0.070584       0.037329  
440080       0.019394  0.021047  0.018437  0.326814       0.614308  
440081       0.012307  0.022301  0.016223  0.896436       0.052732  

[440082 rows x 7 columns]

In [5]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel


def pool(hidden_state, mask, pooling_method="mean"):
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]

inputs = [
    # 
    "paraphrase: В Ярославской области разрешили работу бань, но без посетителей",
    "categorize_entailment: Женщину доставили в больницу, за ее жизнь сейчас борются врачи.",
    "search_query: Сколько программистов нужно, чтобы вкрутить лампочку?",
    # 
    "paraphrase: Ярославским баням разрешили работать без посетителей",
    "categorize_entailment: Женщину спасают врачи.",
    "search_document: Чтобы вкрутить лампочку, требуется три программиста: один напишет программу извлечения лампочки, другой — вкручивания лампочки, а третий проведет тестирование."
]

tokenizer = AutoTokenizer.from_pretrained("sergeyzh/BERTA")
model = AutoModel.from_pretrained("sergeyzh/BERTA").to('cuda')

tokenized_inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors="pt").to('cuda')

with torch.no_grad():
    outputs = model(**tokenized_inputs)
    
embeddings = pool(
    outputs.last_hidden_state, 
    tokenized_inputs["attention_mask"],
    pooling_method="mean"
)

embeddings = F.normalize(embeddings, p=2, dim=1)

In [12]:
class TextData(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx]

In [13]:
dl = torch.utils.data.DataLoader(TextData(reviews['text'].values), batch_size=32, shuffle=False)

In [24]:
from tqdm.auto import tqdm

embs = []
for text in tqdm(dl):
    tokenized_inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors="pt").to('cuda')
    
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
        
    embeddings = pool(
        outputs.last_hidden_state, 
        tokenized_inputs["attention_mask"],
        pooling_method="mean"
    )
    embs.append(embeddings)

  0%|          | 0/13753 [00:00<?, ?it/s]

In [25]:
embeddings = F.normalize(torch.cat(embs).cpu(), p=2, dim=1)

In [30]:
np.save('berta_embs_reviews.npy', embeddings.numpy())